In [1]:
import pandas as pd

import codecs

import folium
import folium.plugins as plugins

from math import log

import warnings
warnings.filterwarnings("ignore")

In [2]:
def popup_html(data):
    html = '<font face="Helvetica" weight=400><h3>{}</h3>'.format(data.region)
    html += '<b>{}:</b> {} чел</p>'.format(name, data.value)
    return html

In [3]:
filename = 'data_cfo.xlsx'
df = pd.read_excel(filename)
df

,region,value
0,Белгородская область,1549151
1,Брянская область,1192491
2,Владимирская область,1358416
3,Воронежская область,2324205
4,Ивановская область,997135
5,Калужская область,1002575
6,Костромская область,633385
7,Курская область,1104008
8,Липецкая область,1139371
9,Московская область,7690863


In [4]:
lg = []
max_log = log(df.value.max())

for index in df.index:
    lg.append(max_log - log(df.loc[index].value))
    
df['log'] = lg

In [5]:
m = folium.Map([55, 39], zoom_start=6, tiles='cartodbpositron', control_scale=True)

filename = r'J:\~ 09_Машинное обучение_Прогноз показателей СЭР\Геограницы\Обновленные\ЦФО.geojson'
name = "Численность населения"

folium.Choropleth(
    geo_data=open(filename, encoding='utf-8').read(),
    name=name,
    data=df,
    key_on='feature.properties.name',
    columns=['region', 'log'],
    fill_color='RdYlGn',
    line_weight=1,
    fill_opacity=0.5,
    line_opacity=0.2,
    popup = ['selsoviet']
).add_to(m)

for index in df.index:
    filename = r'J:\~ 09_Машинное обучение_Прогноз показателей СЭР\Геограницы\Обновленные\ЦФО\{}.geojson'.format(df.loc[index].region)
    geoj = folium.GeoJson(data=open(filename, encoding='utf-8').read(), control=False,
                          style_function = lambda x: {'color': 'black', 'fillOpacity' : 0, 'opacity' : 0.1, 'weight': 1},
                          highlight_function=lambda x: {'color': 'black', 'fillOpacity' : 0.25, 'opacity' : 0.1, 'weight': 1},
                          smooth_factor=0)
    iframe = folium.IFrame(popup_html(df.loc[index]), width=250, height=100)
    popup = folium.Popup(iframe)
    popup.add_to(geoj)
    geoj.add_to(m)

plugins.Fullscreen(position='topleft', title='На весь экран', title_cancel='Уменьшить', force_separate_button=True).add_to(m)

m.save('{} карта по ЦФО.html'.format(name))